In [1]:
include("./DroneLib.jl")

step! (generic function with 1 method)

In [ ]:
# function get_state(drone::Drone)
#     return [drone.position..., drone.velocity..., drone.orientation..., drone.angular_velocity...]
# end

# function apply_changes!(drone::Drone, du)
#     # This function should update the drone's state based on the changes calculated by drone_dynamics!.
#     # For example:
#     drone.position .+= du[1:3]
#     drone.velocity .+= du[4:6]
#     drone.orientation .+= du[7:9]
#     drone.angular_velocity .+= du[10:12]
# end

# function ReinforcementLearningBase.step!(env::DroneEnv, action)
#     # Apply the action (change in RPMs or rotor forces), then update drone's state.
#     # Use the drone_dynamics! function here.
#     du = zeros(12) #assuming the size of du matches with the state size
#     drone_dynamics!(du, get_state(env.drone), (t) -> action, 0.0) # assuming that drone_dynamics! is defined in a way that is compatible with this use

#     # Update drone state with the changes
#     apply_changes!(env.drone, du)

#     # Also update the reward and done status.
#     env.reward = reward(env, action)
#     env.done = is_done(env)
# end

In [ ]:
# env = DroneEnv()

# total_reward = 0.0
# num_steps = 0

# trajectory = []

# # Start the episode
# while !RLBase.is_terminated(env)
#     # Select a random action
#     action = rand(RLBase.action_space(env))

#     # Apply the action to the environment
#     reward, done = env(action)
# 	# println("Reward: ", reward)

#     # Update the total reward and number of steps
#     total_reward += reward
#     num_steps += 1

#     # Save the drone's position
#     push!(trajectory, env.drone.position)

#     if done
#         break
#     end
# end
# println("Episode finished after ", num_steps, " steps. Total reward: ", total_reward)
# println("Final position: ", env.drone.position)

# ### 3d Plot the trajectory
# x = [trajectory[i][1] for i in 1:length(trajectory)]
# y = [trajectory[i][2] for i in 1:length(trajectory)]
# z = [trajectory[i][3] for i in 1:length(trajectory)]

# # plotlyjs()
# plot(x, y, z, label="trajectory", xlabel="x", ylabel="y", zlabel="z", title="Drone Trajectory", legend=:bottomright)


In [ ]:
# ### Start the reinforcement learning
# ### Implement a Deep learning model for Q-learning algorithm to learn the optimal policy
# ### The state space is the drone's position and velocity
# ### The action space is the change in RPMs of the propellers
# ### The reward function is the distance to the target
# ### The termination condition is when the drone reaches the target or crashes

# ### Define the Q-network
# struct QNetwork
# 	env::DroneEnv
# end

# ### Define the DeepQLearningPolicy
# struct DeepQLearningPolicy
# 	env::DroneEnv
# 	Q::QNetwork
# 	epsilon::Float64
# 	gamma::Float64
# 	alpha::Float64
# 	epsilon_decay::Float64
# 	gamma_decay::Float64
# 	alpha_decay::Float64
# 	epsilon_min::Float64
# 	gamma_min::Float64
# 	alpha_min::Float64
# end

# policy = DeepQLearningPolicy(env, QNetwork(env), 0.1, 0.99, 0.1, 0.1, 0.1, 0.1, 0.1, 0.95, 0.1)

# ### Define the Q-learning algorithm
# function QLearning(policy::DeepQLearningPolicy, num_episodes::Int64)
# 	# Initialize the Q-network
# 	Q = policy.Q
# 	# Initialize the total reward
# 	total_reward = 0.0
# 	# Initialize the number of steps
# 	num_steps = 0
# 	# Initialize the trajectory
# 	trajectory = []
# 	# Initialize the episode
# 	for i in 1:num_episodes
# 		# Initialize the environment
# 		env = policy.env
# 		# Initialize the episode
# 		while !RLBase.is_terminated(env)
# 			# Select an action
# 			action = rand(RLBase.action_space(env))
# 			# Apply the action to the environment
# 			reward, done = env(action)
# 			# Update the total reward and number of steps
# 			total_reward += reward
# 			num_steps += 1
# 			# Save the drone's position
# 			push!(trajectory, env.drone.position)
# 			# Update the Q-network
# 			Q = update(Q, env, action, reward, done)
# 			# Update the policy
# 			policy = update(policy, Q)
# 			if done
# 				break
# 			end
# 		end
# 	end
# 	return policy, Q, total_reward, num_steps, trajectory
# end

# ### Define the update function for the Q-network
# function update(Q::QNetwork, env::DroneEnv, action, reward, done)
# 	# Update the Q network based on the reward and done status
	
# end

In [ ]:
### Implement the PPO algorithm
### Define the PPO policy
mutable struct PPOPolicy
	env::DroneEnv
	policy::Chain
	epsilon::Float64
	gamma::Float64
	alpha::Float64
	epsilon_decay::Float64
	gamma_decay::Float64
	alpha_decay::Float64
	epsilon_min::Float64
	gamma_min::Float64
	alpha_min::Float64
end

# function PPOPolicy(env::DroneEnv, policy::Chain, epsilon::Float64, gamma::Float64, alpha::Float64, epsilon_decay::Float64, gamma_decay::Float64, alpha_decay::Float64, epsilon_min::Float64, gamma_min::Float64, alpha_min::Float64)
# 	return PPOPolicy(env, policy, epsilon, gamma, alpha, epsilon_decay, gamma_decay, alpha_decay, epsilon_min, gamma_min, alpha_min)
# end

In [ ]:
function gaussian_policy(network::Chain, state::Array{Float64,1})
	mu, log_sigma = network(state)  # Assume the network outputs means and log standard deviations
    sigma = exp.(log_sigma)
    return MvNormal(mu, sigma)  # Return a multivariate normal distribution
end

In [ ]:
function prob(policy::PPOPolicy, state::Array{Float64,1}, action::Array{Float64,1})
	distribution = gaussian_policy(policy.policy, state)
    return pdf(distribution, action)  # Probability density function for the given action
end

In [ ]:
function get_action(policy::PPOPolicy, state::Array{Float64,1})
	distribution = gaussian_policy(policy.policy, state)
	action = rand(distribution)
	return action
end

In [ ]:
### Define the update function for the PPO policy
function update(policy::PPOPolicy, current_batch, optimizer)
	for (state, action, reward, new_state, done) in current_batch
		value = reward + policy.gamma * (done ? 0.0 : prob(policy, new_state, get_action(policy, state)))
		advantage = value - prob(policy, state, action)
		ratio = prob(policy, state, action) / prob(policy, state, action)
		loss = -min(ratio * advantage, clamp(ratio, 1 - policy.epsilon, 1 + policy.epsilon) * advantage)
		grads = gradient(() -> loss, params(policy.policy))
		Flux.Optimise.update!(optimizer, params(policy.policy), grads)
	end
end

# function update(policy::PPOPolicy, current_batch, optimizer)
#     for (state, action, reward, new_state, done) in current_batch
#         # value = estimate_value(policy, state)  # Use value function to estimate state value
#         # old_prob = prob(policy, state, action)  # Probability under the previous policy

#         new_prob = prob(policy, new_state, get_action(policy, new_state))  # Probability under the current policy
#         ratio = new_prob / old_prob

#         advantage = reward + policy.gamma * (done ? 0.0 : value) - value  # Compute advantage using value function

#         surrogate1 = ratio * advantage
#         surrogate2 = clamp(ratio, 1 - policy.epsilon, 1 + policy.epsilon) * advantage
#         loss = -min(surrogate1, surrogate2)

#         Flux.Optimise.update!(optimizer, params(policy.policy)) do
#             grads = gradient(() -> loss, params(policy.policy))
#             grads = Flux.Tracker.update!(optimizer, grads)
#             return grads
#         end
#     end
# end

In [ ]:
### Define the PPO algorithm
function PPO(policy::PPOPolicy, num_episodes::Int64, batch_size::Int64)
	# Initialize the total reward
	total_reward = Base.Threads.Atomic{Float64}(0.0)
	# Initialize the number of steps
	num_steps = Base.Threads.Atomic{Int64}(0.0)
	# Initialize the trajectory
	trajectories = []
	optimizer = ADAM(0.01)

	# Initialize thread-local buffers
	# local_batches = Vector{Tuple{Array{Float64,1}, Array{Float64,1}, Float64, Array{Float64,1}, Bool}}[]

	# Initialize the episode
	@threads for i in 1:num_episodes
		# Initialize the environment
		env = policy.env
		trajectory = []
		local_batch = []

		# Reset the environment
		RLBase.reset!(env)

		# Initialize the episode
		while !RLBase.is_terminated(env)
			state = RLBase.state(env)
			action = get_action(policy, state)

			# Apply the action to the environment
			reward, done = step!(env, action)
			# Update the total reward and number of steps
			Base.Threads.atomic_add!(total_reward, reward)
			Base.Threads.atomic_add!(num_steps, 1)
			# Save the drone's position
			push!(trajectory, env.drone.position)

			push!(local_batch, (state, action, reward, state, done))

			# Update the policy if batch size is reached
			if length(local_batch) >= batch_size
				update(policy, local_batch, optimizer)
				local_batch = []
			end

			if done
				break
			end
		end

		update(policy, local_batch, optimizer)

		push!(trajectories, trajectory)

		# Decay the exploration
		policy.epsilon = max(policy.epsilon * policy.epsilon_decay, policy.epsilon_min)
	end

	return policy, total_reward, num_steps, trajectories
end

In [ ]:
target_pos = Array([1.0, 1.0, 1.0])
env = DroneEnv(target_pos)
### Create the PPO policy# Define the policy network
policy_chain = Chain(
    Dense(16, 64, relu),
    Dense(64, 64, relu),
    Dense(64, 8),  # Assume your state is 4-dimensional, and you output mean and log standard deviation for 4-dimensional action
    x -> (x[1:4], x[5:8])  # Split the output into two parts,
    # the first part is the mean, the second part is the log standard deviation
)

policy = PPOPolicy(env, policy_chain, 0.1, 0.99, 0.1, 0.1, 0.1, 0.1, 0.1, 0.95, 0.1)

In [ ]:
### Run the PPO algorithm
for i in 1:100
	policy, total_reward, num_steps, trajectory = PPO(policy, 5000, 16)
	println("Episode $i: Total reward: $total_reward, Number of steps: $num_steps")
end

In [ ]:
for j in 1:length(trajectory)
	traj = trajectory[j]
	x = [traj[i][1] for i in 1:length(traj)]
	y = [traj[i][2] for i in 1:length(traj)]
	z = [traj[i][3] for i in 1:length(traj)]
	### Plot the trajectory
	plot(x, y, z, label="trajectory", xlabel="x", ylabel="y", zlabel="z", title="Drone Trajectory $j", legend=:bottomright)
	### Save the plot
	savefig("trajectories/trajectory-$j.png")
# plot!(x, y, z)
### Plot the drone
# scatter!([x[end]], [y[end]], [z[end]], label="")
end

In [ ]:
total_reward

In [ ]:
### Run a sample Episode
env = DroneEnv([1.0, 1.0, 1.0])

total_reward = 0.0
num_steps = 0

trajectory = []

while !RLBase.is_terminated(env)
	# Select a random action
	action = get_action(policy, RLBase.state(env))

	# Apply the action to the environment
	reward, done = step!(env, action)
	# println("Reward: ", reward)

	# Update the total reward and number of steps
	total_reward += reward
	num_steps += 1

	# Save the drone's position
	push!(trajectory, env.drone.position)

	if done
		break
	end
end
println("Episode finished after ", num_steps, " steps. Total reward: ", total_reward)
println("Final position: ", env.drone.position)

In [ ]:
struct PIDController
    kp::Float64
    ki::Float64
    kd::Float64
    integral::Array{Float64, 1}
    prev_error::Array{Float64, 1}
end

function PIDController(kp::Float64, ki::Float64, kd::Float64)
    integral = zeros(3)
    prev_error = zeros(3)
    PIDController(kp, ki, kd, integral, prev_error)
end

In [ ]:
function update_pid!(pid::PIDController, error::Array{Float64, 1})
    pid.integral += error
    derivative = error - pid.prev_error
    pid.prev_error = error

    pid_output = pid.kp .* error + pid.ki .* pid.integral + pid.kd .* derivative
    return pid_output
end

In [ ]:
function get_random_action() 
	base_rpm = 7900.0
	action = [base_rpm, base_rpm, base_rpm, base_rpm]
	# action = action + rand(4) * 1000
	return action
end



In [ ]:
### Create a PID Control for the episode

env = DroneEnv([1.0, 1.0, 1.0], 10.0)
env.action = [7500.0, 7500.0, 7500.0, 7500.0]
total_reward = 0.0
num_steps = 0

trajectory = []

while !RLBase.is_terminated(env)
	# Select a random action
	action = PIDControlDroneEnvAction(env)
	# action = get_random_action()
	# if num_steps > 95
	# 	action = [0.0, 0.0, 0.0, 0.0]
	# end
	# Apply the action to the environment
	reward, done = step!(env, action)
	# println("Reward: ", reward)

	# Update the total reward and number of steps
	total_reward += reward
	num_steps += 1

	# Save the drone's position
	push!(trajectory, env.drone.position)

	if done
		break
	end
end

println(env.drone.position, env.drone.velocity, env.drone.angular_velocity)

In [ ]:
a = 10

a += -9.81

In [ ]:
println(a)